# LIDOQA: LIDO Quality Assessment

## Introduction

Heterogeneity of asset management systems, export worflows, cataloguing standards, objects results in heterogeneity of LIDO data.
Still, some general rules can be formulated that make LIDO data better.

Some basic guidelines for 'good' LIDO-Data:

- good LIDOs describe objects carefully und comprehensively
- good LIDO is both human and machine-readable
- good LIDO uses controlled vocabulary and complies with good practices of LOD (use URIs wherever possible)

In [1]:
from glob import glob
from lxml import etree
from collections import Counter, defaultdict
from urllib.parse import urlparse
import requests

In [2]:
# LIDO  Namespace für registrieren
NSMAP = {
    'lido' : 'http://www.lido-schema.org'
}

## Ist die LIDO-XML 'schemavalide'?

Für die optimale Verwendung von XML-Dateien müssen diese oftmals einer möglichst standardisierten Struktur folgen.
Diese Struktur kann in einer [XML Schema Definition Language (XSD)](https://www.w3.org/TR/xmlschema11-1/) beschrieben werden.
Auch für das LIDO-Schema werden XSD-Dateien bereitgestellt:

| LIDO-Version | URL |
|:-- |:--- |
| 1.0 | <http://www.lido-schema.org/schema/v1.0/lido-v1.0.xsd> |
| 1.1 | <http://lido-schema.org/schema/v1.1/lido-v1.1.xsd> |

Es empfiehlt sich, generierte XML-Dateien auf Schemavalidität zu prüfen, d.h. sicherzustellen, dass die XML-Datei den Vorgaben aus der XSD-Schemadatei entspricht.

Für die Prüfung bietet sich das [Python-Modul `lxml`](https://lxml.de/2.1/validation.html#xmlschema) an.

### Erstellung eines  `XMLSchema`-Objekts

Zunächst muss die XSD-Datei eingelesen und in ein [`XMLSchema`-Objekt](https://lxml.de/api/lxml.etree.XMLSchema-class.html) umgewandelt werden.

In [3]:
#lidoxsd = "http://lido-schema.org/schema/v1.1/lido-v1.1.xsd"
lidoxsd = "http://www.lido-schema.org/schema/v1.0/lido-v1.0.xsd"
res = requests.get(lidoxsd)
xmlschema_doc = etree.fromstring(res.content)
xmlschema = etree.XMLSchema(xmlschema_doc)
# Falls das Schema lokal vorliegt, kann der Dateipfad auch über ein Argument 'file' übergeben werden
# xmlschema = etree.XMLSchema(file="lido-v1.0.xsd")

### Validierung

Für die Validierung an sich gibt es nun mehrere Möglichkeiten.
Das zu validierende XML-Dokument muss dabei jedoch zunächste immer mit `etree.parse(<XML-DATEINAME>)` (oder wenn die XML-Datei als String vorliegt: `etree.fromstring(<XML-STRING>)`) geparset, d.h. zu einem `ElementTree`-Objekt umgewandelt werden.

Im Folgenden sollen alle LIDO-Dateien des `lido`-Ordners auf Schemavalidität geprüft werden.

Gezeigt werden verschiedene Methoden der `XMLSchema`-Klasse.

In [4]:
lidos = glob('lido/*xml')

#### Überprüfung mit `.validate()`

Die Methode `.validate()` liefert einen Boole'schen Wahrheitswert (`True` bei validem LIDO /`False` bei nicht validem LIDO) zurück.

Eine Liste fehlerhafter Dateien ließe sich also folgendermaßen erzeugen (nähere Informationen zu den Fehlern bietet die `.error_log`-Property des `XMLSchema`-Objekts):

In [13]:
invalidLidos = []
validLidos = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    if xmlschema.validate(tree) is False:
        invalidLidos.append(LIDO)
    else:
        validLidos.append(LIDO)

#### Überprüfung mit Fehlermeldung

Während `.validate()` einen Wahrheitswert zurückgibt, führen `.assert_` und `.assertValid` zu Fehlern (`AssertionError` bzw. `DocumentInvalid`), die ggf. über `try` und `except` abgefangen werden müssen.
Die Fehlermeldungen enthalten auch nähere Informationen zum jeweiligen Fehler.
Diese sind auch im `error_log` gespeichert.

In [147]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    try:
        xmlschema.assert_(tree)
        #xmlschema.assertValid(tree)
    except Exception as e:
        print(e)

Element '{http://www.lido-schema.org}roleActor': This element is not expected. Expected is ( {http://www.lido-schema.org}actor )., line 70


### Interpretation des Validierungsergebnisses

Die Interpretation der Fehlermeldungen der Validierung ist nicht trivial.
Sie setzt eine genauere Auseinandersetzung mit der XSD-Datei bzw. der Spezifikation (LIDO [v.1.0](http://www.lido-schema.org/schema/v1.0/lido-v1.0-specification.pdf)/[v.1.1](https://lido-schema.org/schema/v1.1/lido-v1.1.html)) voraus.

#### Eine kurze Leseanleitung für die LIDO-XSD-Datei

In [6]:
NSMAP.update({'xsd': 'http://www.w3.org/2001/XMLSchema'})

In [7]:
for ct in xmlschema_doc.findall('.//xsd:complexType', NSMAP):
    print(ct.tag,ct.attrib)

{http://www.w3.org/2001/XMLSchema}complexType {'name': 'actorComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'actorInRoleComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'actorInRoleSetComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'actorSetComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'administrativeMetadataComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'appellationComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'conceptComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {'name': 'dateComplexType'}
{http://www.w3.org/2001/XMLSchema}complexType {}
{http://www.w3.

In [12]:
for seq in xmlschema_doc.findall('.//xsd:sequence', NSMAP):
    print(seq.tag,seq.attrib)
    for x in seq.iter('{http://www.w3.org/2001/XMLSchema}element'):
        print(x.tag,x.text,x.attrib)
    input()

{http://www.w3.org/2001/XMLSchema}sequence {}
{http://www.w3.org/2001/XMLSchema}element 
				 {'name': 'actorID', 'type': 'lido:identifierComplexType', 'minOccurs': '0', 'maxOccurs': 'unbounded'}
{http://www.w3.org/2001/XMLSchema}element 
				 {'name': 'nameActorSet', 'type': 'lido:appellationComplexType', 'maxOccurs': 'unbounded'}
{http://www.w3.org/2001/XMLSchema}element 
				 {'name': 'nationalityActor', 'minOccurs': '0', 'maxOccurs': 'unbounded'}
{http://www.w3.org/2001/XMLSchema}element 
				 {'name': 'vitalDatesActor', 'type': 'lido:dateComplexType', 'minOccurs': '0'}
{http://www.w3.org/2001/XMLSchema}element 
				 {'name': 'genderActor', 'type': 'lido:textComplexType', 'minOccurs': '0', 'maxOccurs': 'unbounded'}


KeyboardInterrupt: Interrupted by user

`sequence`-Knoten enhalten `element`-Knoten.
Diese müssen in der angegebenen Reihenfolge erscheinen.
Das Attribut `name` gibt den Namen des Knotens im LIDO-XML-an.


In [146]:
for LIDO in lidos[:2]:
    tree = etree.parse(LIDO)
    xmlschema.assertValid(tree)
    try:
        #xmlschema.assert_(tree)
        xmlschema.assertValid(tree)
    except Exception as e:
        print(e)
        print(xmlschema.error_log)
        print(LIDO)
        input()

DocumentInvalid: Element '{http://www.lido-schema.org}roleActor': This element is not expected. Expected is ( {http://www.lido-schema.org}actor )., line 70

In [137]:
xmlschema.error_log

lido/KG7FJ2CWIVQXFM4D3PAGVMJS5WB6SXY4.xml:70:0:ERROR:SCHEMASV:SCHEMAV_ELEMENT_CONTENT: Element '{http://www.lido-schema.org}roleActor': This element is not expected. Expected is ( {http://www.lido-schema.org}actor ).

In [148]:
invalid = []
for LIDO in lidos:
    ID = LIDO.replace('lido/','').replace('.xml','')
    tree = etree.parse(LIDO)
    print(type(tree))
    instit = tree.find('//lido:repositorySet//lido:legalBodyName/lido:appellationValue', NSMAP).text
    #print(tree.find('//lido:roleActor...', NSMAP))
    if xmlschema.validate(tree) is False:
        log = xmlschema.error_log
        for x in log:
            invalid.append({
                'ID' : ID,
                'line' : x.line,
                'path' : x.path,
                'message' : x.message
            })
    #print(dir(xmlschema))
    #print(xmlschema.validate(tree))
    #print(dir(xmlschema.error_log))
    #print(xmlschema.error_log.last_error)
    
    #try:
    #    xmlschema.assertValid(tree)    
    #except Exception as e:
    #    print(e)
    #    invalid.append(instit)

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

...

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.etree._ElementTree'>
<class 'lxml.e

In [149]:
with open('LIDO_error_log.csv','w') as OUT:
    writer = csv.DictWriter(OUT, fieldnames = list(invalid[0].keys()))
    writer.writeheader()
    writer.writerows(invalid)

In [ ]:
xmlschema.error_log

## Eregnisse sammeln


In [61]:
eventtypes = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for event in tree.findall('//lido:eventType/lido:term', NSMAP):
        eventtypes.append(event.text)
        

In [63]:
Counter(eventtypes)

Counter({'Herstellung': 15013,
         'Production': 33625,
         'Event (non-specified)': 35319,
         'Aufgenommen': 620,
         'Hergestellt': 761,
         'Fund': 67,
         'Aufführung': 502,
         'Vorlagenerstellung': 10,
         'Geistige Schöpfung': 117,
         'Veröffentlichung': 126,
         'Gezeichnet': 139,
         'Restauriert': 38,
         'Druckplatte hergestellt': 17,
         'Verkauft': 10,
         'Form entworfen': 5,
         'Veröffentlicht': 29,
         'Gefunden': 49,
         'Gemalt': 14,
         'Gekauft': 3,
         'Besessen': 7,
         'Bearbeitet': 1,
         'Provenienz': 6,
         'Bearbeitung': 1,
         'Schenkung': 1,
         'Auftrag': 1,
         'Versteigert': 2,
         'Nutzung': 5,
         'Gedruckt': 3})

In [17]:
sources = []
for LIDO in lidos:
    tree = etree.parse(LIDO)
    sources.extend([x.text for x in tree.findall('//*[@lido:source]', NSMAP)])

In [22]:
c = Counter(sources).most_common()

In [29]:
sourcesFreq = Counter([urlparse(s).netloc for s in sources]).most_common()[:-20:-1]

In [30]:
sourcesFreq

[('www.lightandglass.eu', 1),
 ('provenienz.gbv.de', 1),
 ('www.hdbg.eu', 1),
 ('www.porzellankompass.de', 1),
 ('www.uhrmacherverzeichnis.de', 1),
 ('slaskaporcelana.pl', 1),
 ('www.cidoc-crm.org', 1),
 ('de.wiktionary.org', 1),
 ('www.wirtschaftslexikon24.com', 1),
 ('watch-wiki.org', 1),
 ('www.mindat.org', 1),
 ('www.uni-stuttgart.de', 1),
 ('www.germandesigners.net', 1),
 ('www.rheinische-geschichte.lvr.de', 2),
 ('www.aat-deutsch.de', 2),
 ('es.wikipedia.org', 2),
 ('steglitz-museum.de', 2),
 ('runeberg.org', 2),
 ('www.stadtteilzeitung-schoeneberg.de', 4)]

In [39]:
controlled_vocabularies = defaultdict(set)
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for x in tree.findall('//*[@lido:source]', NSMAP):
        controlled_vocabularies[x.attrib.get('{http://www.lido-schema.org}source')].add(x.tag)

In [53]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    hit = tree.find('//*[@lido:source="Wikipedia"]', NSMAP)
    if hit is not None:
        print(hit.tag,hit.text,hit.attrib)
        print(LIDO)
        input()


{http://www.lido-schema.org}conceptID https://de.wikipedia.org/wiki/Musiker {'{http://www.lido-schema.org}source': 'Wikipedia', '{http://www.lido-schema.org}type': 'uri'}
lido/ZMKN5GYCUCHK4X3HZM4JSBIK6D7VFYFG.xml

{http://www.lido-schema.org}conceptID https://de.wikipedia.org/wiki/Theater {'{http://www.lido-schema.org}source': 'Wikipedia', '{http://www.lido-schema.org}type': 'uri'}
lido/JRH5LJ7NJWWGBD3PG6TFCCGF7J5FHI3J.xml

{http://www.lido-schema.org}actorID https://de.wikipedia.org/wiki/Ludwig_Burger {'{http://www.lido-schema.org}source': 'Wikipedia', '{http://www.lido-schema.org}type': 'uri'}
lido/JIVPU3WA23FS6TAWPYJY64OZ36HEEC5A.xml

{http://www.lido-schema.org}actorID https://sv.wikipedia.org/wiki/Tyska_R%C3%B6da_korset {'{http://www.lido-schema.org}source': 'Wikipedia', '{http://www.lido-schema.org}type': 'uri'}
lido/R6KBMWZM3A4RXGRXUZMUO2JM573T6RHF.xml

{http://www.lido-schema.org}conceptID https://de.wikipedia.org/wiki/Uniform {'{http://www.lido-schema.org}source': 'Wikipedia',

KeyboardInterrupt: Interrupted by user

In [15]:
Counter([r.tag for r in results])

Counter({'{http://www.lido-schema.org}legalBodyID': 440,
         '{http://www.lido-schema.org}conceptID': 595,
         '{http://www.lido-schema.org}placeID': 194,
         '{http://www.lido-schema.org}lidoRecID': 25,
         '{http://www.lido-schema.org}actorID': 27,
         '{http://www.lido-schema.org}resourceID': 2})

In [21]:
Counter([r.attrib.get('{http://www.lido-schema.org}source') for r in results if r.tag == "{http://www.lido-schema.org}conceptID" ])

Counter({'http://d-nb.info/gnd': 141,
         'http://vocab.getty.edu/aat/': 70,
         'http://terminology.lido-schema.org/eventType': 137,
         'http://creativecommons.org': 197,
         'local': 2,
         'aat': 5,
         'gnd': 6,
         'iconclass': 1,
         'lcsh': 6,
         'wikidata': 7,
         'Wikipedia': 7,
         'md:term': 10,
         'grobsystematik': 2,
         'oberbegriffsdatei': 2,
         'http://terminology.lido-schema.org': 2})

In [25]:
for r in results:
    print(r.attrib.get('{http://www.lido-schema.org}source'))
    print(r.text)
    print(r.tag)

ISIL (ISO 15511)
info:isil/DE-MUS-061525
{http://www.lido-schema.org}legalBodyID
ISIL (ISO 15511)
info:isil/DE-MUS-061525
{http://www.lido-schema.org}legalBodyID
http://d-nb.info/gnd
http://d-nb.info/gnd/4149435-0
{http://www.lido-schema.org}conceptID
http://vocab.getty.edu/aat/
http://vocab.getty.edu/aat/300128371
{http://www.lido-schema.org}conceptID
ISIL (ISO 15511)
http://ld.zdb-services.de/resource/organisations/DE-2612
{http://www.lido-schema.org}legalBodyID
http://d-nb.info/gnd/
http://d-nb.info/gnd/4005728-8
{http://www.lido-schema.org}placeID
https://www.geonames.org/
https://www.geonames.org/2950159/berlin.html
{http://www.lido-schema.org}placeID
http://terminology.lido-schema.org/eventType
http://terminology.lido-schema.org/lido00007
{http://www.lido-schema.org}conceptID
http://d-nb.info/gnd
http://d-nb.info/gnd/4045893-3
{http://www.lido-schema.org}conceptID
http://terminology.lido-schema.org/eventType
http://terminology.lido-schema.org/lido00003
{http://www.lido-schema.org

In [29]:
sources = defaultdict(set)

In [30]:
for r in results:
    sources[r.attrib.get('{http://www.lido-schema.org}source')].add(r.tag)

In [55]:
cv_by_category = defaultdict(set)

for k,v in sources.items():
    for V in v:
        cv_by_category[V].add(k)

In [56]:
cv_by_category

defaultdict(set,
            {'{http://www.lido-schema.org}legalBodyID': {'ISIL (ISO 15511)',
              'ISL (ISO 15511)'},
             '{http://www.lido-schema.org}conceptID': {'AAT',
              'CIDOC-CRM',
              'Wikipedia',
              'aat',
              'ackerbau',
              'allgemein',
              'bne',
              'bnf',
              'gnd',
              'grobsystematik',
              'http://creativecommons.org',
              'http://d-nb.info/gnd',
              'http://terminology.lido-schema.org',
              'http://terminology.lido-schema.org/eventType',
              'http://vocab.getty.edu/aat/',
              'iconclass',
              'lcsh',
              'local',
              'md:term',
              'mindat.org',
              'moebeltypologie',
              'ndl',
              'oberbegriffsdatei',
              'wikidata'},
             '{http://www.lido-schema.org}actorID': {'GND',
              'NDB/ADB',
              'ULAN'

In [58]:
for LIDO in lidos:
    tree = etree.parse(LIDO)
    for x in tree.findall('//lido:legalBodyID[@lido:source="ISL (ISO 15511)"]', NSMAP):
        print(x.tag,x.attrib,x.text)
        print(LIDO)

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/2VK2HGMN7X6Q4STQ6H6525HOFRZNPHEF.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/2VK2HGMN7X6Q4STQ6H6525HOFRZNPHEF.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/OW2NHE2BNCQMI74PSBUXCHPQL3NWUVRF.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/OW2NHE2BNCQMI74PSBUXCHPQL3NWUVRF.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/DE

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/WGB4VJ3XWDLJID6YD4OBC4JRESAOYWYS.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/WGB4VJ3XWDLJID6YD4OBC4JRESAOYWYS.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/NM54ZXAFDOQF46XSZ3GXERMWPTEG5NHC.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/NM54ZXAFDOQF46XSZ3GXERMWPTEG5NHC.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/EY

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/73QTMBG2HKME3MDEQ47S4AL4GDM5WZBX.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/73QTMBG2HKME3MDEQ47S4AL4GDM5WZBX.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/QKEWDR65W4ZDDXXIX7DVLA43XS7GHXJB.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/QKEWDR65W4ZDDXXIX7DVLA43XS7GHXJB.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/7Q

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/FNKIIIE2LZ6EOWPFXJP2AW3NUIV3TW64.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/FNKIIIE2LZ6EOWPFXJP2AW3NUIV3TW64.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/UW4Q62DGICMPH5UMM3TML5CUYNBP3MLM.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/UW4Q62DGICMPH5UMM3TML5CUYNBP3MLM.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/SR

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/ZOQ2QQJPGFE5CSAAP7OBGSFEDRGOTEMG.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/ZOQ2QQJPGFE5CSAAP7OBGSFEDRGOTEMG.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/DRL7PR7ZWFB3B55FIJ4YT4LYD7SVSDCK.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/DRL7PR7ZWFB3B55FIJ4YT4LYD7SVSDCK.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/H4

...

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/G7KOCBASPQV5CZ5VVBBXQBQLHPUWVAWH.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/G7KOCBASPQV5CZ5VVBBXQBQLHPUWVAWH.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/4RDNLTWXNQTFNK6SVKD5QYG64RUL7DHX.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/4RDNLTWXNQTFNK6SVKD5QYG64RUL7DHX.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/CK

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/54Q4PGRWB3LSAWREV4XTI7MDDCBAU4M2.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/54Q4PGRWB3LSAWREV4XTI7MDDCBAU4M2.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/QQE5CX72T674LBRVVBMUQQNJYU7OWE6B.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/QQE5CX72T674LBRVVBMUQQNJYU7OWE6B.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/WC

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/EY27OW5KKFDA2VMG3XQG5YNUALFI5LYC.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/EY27OW5KKFDA2VMG3XQG5YNUALFI5LYC.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/6BPFXEV3T4GTXAMG4IAWT72PSHHGEEAB.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/6BPFXEV3T4GTXAMG4IAWT72PSHHGEEAB.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/KO

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/HXO5Q4ISVL3R4MZXRKND7GETYDWG3G77.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/HXO5Q4ISVL3R4MZXRKND7GETYDWG3G77.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/KGIB46A65IKPWKQR6JKTM2LIWZPRYVT4.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/KGIB46A65IKPWKQR6JKTM2LIWZPRYVT4.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/RN

{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/IGA4P7M5MAEQOHSD3WJB2VWSPASJL65U.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/IGA4P7M5MAEQOHSD3WJB2VWSPASJL65U.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/2LB3J2CHCWKQSXKQAVPPFEXGFDIBKM6I.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/2LB3J2CHCWKQSXKQAVPPFEXGFDIBKM6I.xml
{http://www.lido-schema.org}legalBodyID {'{http://www.lido-schema.org}source': 'ISL (ISO 15511)', '{http://www.lido-schema.org}type': 'URI'} info:isil/DE-MUS-275412
lido/TT